# Notebook with Minimal example to run hposuite

In [ ]:
# It is assumed that the datadir is hposuite/data

from lib.benchmarks import BENCHMARKS
from lib.optimizers import OPTIMIZERS

print(BENCHMARKS)
print(OPTIMIZERS)

/home/soham/repos/hposuite/data
{'Ex_Functional_Bench': <hpoglue.benchmark.FunctionalBenchmark object at 0x77edcc1862f0>, 'Ex_Surrogate_Bench': BenchmarkDescription(name='Ex_Surrogate_Bench', config_space=Configuration space object:
  Hyperparameters:
    lr_decay_factor, Type: UniformFloat, Range: [0.010093, 0.989012], Default: 0.4995525
    lr_initial, Type: UniformFloat, Range: [1e-05, 9.779176], Default: 0.0098889716351, on log-scale
    lr_power, Type: UniformFloat, Range: [0.100708, 1.999376], Default: 1.050042
    opt_momentum, Type: UniformFloat, Range: [5.9e-05, 0.998993], Default: 0.0076772773169, on log-scale
, load=functools.partial(<function _get_surrogate_benchmark at 0x77edcc181b40>, benchmark_name='cifar100_wideresnet_2048', datadir=PosixPath('/home/soham/repos/hposuite/data')), metrics={'valid_error_rate': Measure(minimize=True, kind=<Kind.METRIC: 'metric'>, bounds=(0, 1))}, test_metrics=None, costs={'train_cost': Measure(minimize=True, kind=<Kind.COST: 'cost'>, bounds

## Running Random Search on a Functional Benchmark (eg: ACKLEY)

In [ ]:
func_benchmark = BENCHMARKS["Ex_Functional_Bench"]
rs = OPTIMIZERS["RandomSearch"]

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite_demo",
    output_dir="../../hposuite-output",
    optimizers=rs,
    benchmarks=func_benchmark,
    num_seeds=5,
    budget=100,
)

study.optimize(overwrite=True)

## Running a MultiFidelity Optimizer (eg: Hyperband) on a Surrogate Benchmark (eg: PD1 Cifar100 Wide Resnet 2048)

In [2]:
sg_benchmark = BENCHMARKS["Ex_Surrogate_Bench"]
mf_optimizer = OPTIMIZERS["Ex_Multifidelity_Opt"]

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite_demo",
    output_dir="../../hposuite-output",
    optimizers=mf_optimizer,
    benchmarks=sg_benchmark,
    num_seeds=5,
    budget=100,
)

study.optimize(overwrite=True)